In [17]:
import pandas as pd
import numpy as np
import os

path = '../data/test1/ccf_offline_stage1_test_revised.csv'
data = pd.read_csv(path)
dataset = data.copy()

In [18]:
#查看缺失值
print(dataset.isnull().any)
#缺失值比例查看
print(dataset.isnull().sum() / len(dataset))

<bound method NDFrame._add_numeric_operations.<locals>.any of         User_id  Merchant_id  Coupon_id  Discount_rate  Distance   
0         False        False      False          False     False  \
1         False        False      False          False      True   
2         False        False      False          False     False   
3         False        False      False          False     False   
4         False        False      False          False     False   
...         ...          ...        ...            ...       ...   
113635    False        False      False          False     False   
113636    False        False      False          False      True   
113637    False        False      False          False      True   
113638    False        False      False          False     False   
113639    False        False      False          False     False   

        Date_received  
0               False  
1               False  
2               False  
3               False  
4

In [21]:
#将 Distance 中空值填充为-1
dataset['Distance'].fillna(-1, inplace=True)
#将领券时间转换为时间类型
dataset['Date_received'] = pd.to_datetime(dataset['Date_received'], format='%Y%m%d')
#判断优惠券是否为满减类型
dataset['Ismanjian'] = dataset['Discount_rate'].map(lambda x: 1 if ":" in str(x) else 0)
#将优惠券转换为折扣率
dataset['Discount_rate'] = dataset['Discount_rate'].map(lambda x: round(float(x), 5)
if ':' not in str(x)
else
round((float(str(x).split(':')[0]) - float(str(x).split(':')[1])) / float(str(x).split(':')[0]), 5))
#数据打标：优惠券折扣大于等于 8 折的为 1，反之为 0
dataset['Label'] = list(map(lambda x: 1 if x <= 0.8 else 0, dataset['Discount_rate']))
#按照日期转换为领券星期
dataset["Weekday_receive"] = dataset['Date_received'].apply(lambda x: x.isoweekday())
#获取领券月份
dataset['Received_month'] = dataset['Date_received'].apply(lambda x: x.month)
#生成处理后的数据表格
path = './ccf_offline_stage1_test_revised_output'
if not os.path.exists(path):
    os.makedirs(path)
dataset.to_csv(path + '.csv', index=False)